In [ ]:
Файл Test_data.csv, содержащий в себе данные о платежных транзакциях плательщиков и получателей:
PAYERACCOUNTNUMBER - номер счета плательщика
PAYEEACCOUNTNUMBER - номер счета получателя
AMOUNT - сумма операции
DATEIN - дата операци

Описание задачи:

Необходимо написать скрипт, рассчитывающий метрики по транзакциям плательщиков и получателей, используя в качестве инструмента Python.
Общая сумма операций плательщика
Общая сумма операций получателя
Общее количество операций плательщика
Общее количество операций получателя
Общая сумма операций от плательщика x получателю y (для каждой пары плательщик-получатель)
Общая количество операций от плательщика x получателю y (для каждой пары плательщик-получатель)
Общая сумма операций от плательщика x получателю y за дату z (для каждой пары плательщик-получатель и каждой даты)
Общая количество операций от плательщика x получателю y за дату z (для каждой пары плательщик-получатель и каждой даты)
Общее сумма операций за дату z  (для каждой даты)
Общее количество операций за дату z  (для каждой даты)
Максимальная сумма отдельной операции каждого плательщика
Максимальная сумма отдельно операции каждого получателя
Результат расчета вывести в .xls файл.
Составить документацию с техническим описанием принципов / алгоритмов расчета.


In [2]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())



In [3]:
sh = gc.open('Interv')

In [35]:
worksheet1 = gc.open('Interv').sheet1
worksheet2 = gc.open("Interv").get_worksheet(1)
worksheet3 = gc.open("Interv").get_worksheet(2)
worksheet4 = gc.open("Interv").get_worksheet(3)
worksheet5 = gc.open("Interv").get_worksheet(4)
worksheet6 = gc.open("Interv").get_worksheet(5)
worksheet7 = gc.open("Interv").get_worksheet(6)
worksheet8 = gc.open("Interv").get_worksheet(7)
worksheet9 = gc.open("Interv").get_worksheet(8)
worksheet10 = gc.open("Interv").get_worksheet(9)
worksheet11 = gc.open("Interv").get_worksheet(10)
worksheet12 = gc.open("Interv").get_worksheet(11)


In [20]:
#INIT
import pandas as pd
from gspread_dataframe import set_with_dataframe # Берем метод из библиотеки 
#гугловских таблиц, который позволяет писать нам данные из датафрейма

df = pd.read_csv("/content/Test_data.csv", sep=";")
df['AMOUNT'] = df['AMOUNT'].str.replace(',','.') #Цифры с запятой питоном не считываются, поэтому меняем их на точку
df["AMOUNT"] = pd.to_numeric(df["AMOUNT"]) # Переводим колонку в чилсенный формат
print (df)


         PAYERACCOUNTNUMBER     PAYEEACCOUNTNUMBER    AMOUNT             DATEIN
0         40720810251876632      40817810351876632   87000.0   21.08.2020 00:00
1         40720810251876632      40817810351876632   87000.0   22.08.2020 00:00
2         40720810251876887      40817810351876669     150.0   23.08.2020 00:00
3         40720810251876895      40817810351876695   18000.0   24.08.2020 00:00
4         40720810255546632      40817810366576632  178000.0   25.08.2020 00:00
...                     ...                    ...       ...                ...
4448  912221210100010005756  408202810291000043758  500000.0   25.02.2021 12:23
4449   31310121010001100800   20255281029100100801  650400.0   28.02.2021 12:23
4450   31310121010001100800   20255281029100100801  650410.0   28.02.2021 12:23
4451   40825121010001100800   45907281029100100801  641600.0   28.02.2021 12:23
4452   44706121010001100800   31410281029100100801  630100.0  28.02.2021 12:23 

[4453 rows x 4 columns]


In [36]:
#Чистка
def clearing (worksheet): #Не вызываем этот метод в отдельном потоке, так как это API
  range_of_cells = worksheet.range('A1:D5000') # Выбираем диапозон клеток
  for cell in range_of_cells: # Начинаем ходить по каждой
    cell.value = '' #Ставим клетке пустое значение
  worksheet.update_cells(range_of_cells) #Обновляем все это

In [37]:
#Общая сумма операций плательщика
clearing(worksheet1) # Чистим наш лист

res=df.groupby(['PAYERACCOUNTNUMBER'])['AMOUNT'].sum().reset_index() # Группируем по плательщику и поле Amount мы суммируем
#reset index - нужен для того, чтобы обновить индекс. Без этого вызова, api не захочет сохранять данные в таблицу (лист)

set_with_dataframe(worksheet1, res) # записываем результат в отдельный лист



In [43]:
#Общая сумма операций получателя
clearing(worksheet2) # Чистим наш лист

res=df.groupby(['PAYEEACCOUNTNUMBER'])['AMOUNT'].sum().reset_index() # Группируем по получателю и поле Amount мы суммируем
#Про reset index упомяналось в ячейке 37 - строка 5
print (res)
set_with_dataframe(worksheet2, res) # записываем результат в отдельный лист


       PAYEEACCOUNTNUMBER       AMOUNT
0    20202010500230060440     40187.94
1    20202018105002300604  12587240.00
2    20202181050023006044   6800000.00
3    20202710038320100992    999000.00
4    20202809051627000802   7406540.00
..                    ...          ...
440  98053810142254000188    300000.00
441  98055810940000003229    300000.00
442  99901810500230060440   4200000.00
443  99902810300000000005   2997000.00
444  99919710038320100992    999000.00

[445 rows x 2 columns]


In [45]:
#Общее количество операций плательщика
clearing(worksheet3) # Чистим наш лист

res=df['PAYERACCOUNTNUMBER'].value_counts(dropna=True).reset_index() #Мы группируем по плательщику наши данные, после чего с помощью value counts, мы считаем кол-во строк. NA мы не считаем
#Про reset index упомяналось в ячейке 37 - строка 5
res.columns = ['PAYERACCOUNTNUMBER', 'How many times'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet3, res) # записываем результат в отдельный лист


       PAYERACCOUNTNUMBER  How many times
0    40217810300000000003             261
1    40217810142254000188             236
2    20202810400003155991             179
3    40506810400003155991             172
4    40417810142254000188             159
..                    ...             ...
366  40821810500005839311               1
367  44706121010001100800               1
368  40702810300350000166               1
369  40702810200490000728               1
370  40109810400003155991               1

[371 rows x 2 columns]


In [48]:
#Общее количество операций получателя
clearing(worksheet4) # Чистим наш лист
res=df['PAYEEACCOUNTNUMBER'].value_counts(dropna=True).reset_index()

#Про reset index упомяналось в ячейке 37 - строка 5
res.columns = ['PAYEEACCOUNTNUMBER', 'How many times'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet4, res) # записываем результат в отдельный лист

       PAYEEACCOUNTNUMBER  How many times
0    40817810300000000005             262
1    40206810516270008028             122
2    40101810500230060440              82
3    49905810601240000026              69
4    20202810051627000802              67
..                    ...             ...
440  31601810038320100992               1
441  20309281029100100801               1
442     40817810351876669               1
443  40701810038320100992               1
444  40701281029100100120               1

[445 rows x 2 columns]


In [49]:
#Общая сумма операций от плательщика x получателю y (для каждой пары плательщик-получатель)
clearing(worksheet5) # Чистим наш лист
res=df.groupby(['PAYERACCOUNTNUMBER','PAYEEACCOUNTNUMBER'])['AMOUNT'].sum().reset_index()# Группируем теперь по двум полям (важен порядок) и суммируем amount

#Про reset index упомяналось в ячейке 37 - строка 5
print (res)
set_with_dataframe(worksheet5, res) # записываем результат в отдельный лист

         PAYERACCOUNTNUMBER     PAYEEACCOUNTNUMBER     AMOUNT
0      20201810038320100992   30619710038320100992   999000.0
1      20202010400003155991   20202810105002300604  3957000.0
2      20202010400003155991   20202811105002300604   600000.0
3      20202010400003155991   40202810105002300604  7500000.0
4      20202010400003155991   40602810105002300604  1700000.0
...                     ...                    ...        ...
1069   91211810300000000003   99902810300000000005  2997000.0
1070   91217810400003155991   99901810500230060440   600000.0
1071  912221210100010005756  408202810291000043758  9000999.0
1072   91312810000760001770   30123810038320100992   999000.0
1073   99901810500230060440   99901810500230060440  3600000.0

[1074 rows x 3 columns]


In [53]:
#Общая количество операций от плательщика x получателю y (для каждой пары плательщик-получатель)
clearing(worksheet6) # Чистим наш лист

res=df.groupby(['PAYERACCOUNTNUMBER','PAYEEACCOUNTNUMBER']).size().reset_index() # Группируем по полям и узнаем размер датафрейма, который получился

#Про reset index упомяналось в ячейке 37 - строка 5
res.columns = ['PAYERACCOUNTNUMBER', 'PAYEEACCOUNTNUMBER','How many times'] # Изменил названия столбцов

print (res)
set_with_dataframe(worksheet6, res) # записываем результат в отдельный лист

         PAYERACCOUNTNUMBER     PAYEEACCOUNTNUMBER  How many times
0      20201810038320100992   30619710038320100992               1
1      20202010400003155991   20202810105002300604              13
2      20202010400003155991   20202811105002300604               1
3      20202010400003155991   40202810105002300604              13
4      20202010400003155991   40602810105002300604               3
...                     ...                    ...             ...
1069   91211810300000000003   99902810300000000005               3
1070   91217810400003155991   99901810500230060440               1
1071  912221210100010005756  408202810291000043758              27
1072   91312810000760001770   30123810038320100992               1
1073   99901810500230060440   99901810500230060440               6

[1074 rows x 3 columns]


In [54]:
#Общая сумма операций от плательщика x получателю y за дату z (для каждой пары плательщик-получатель и каждой даты)
clearing(worksheet7) # Чистим наш лист

res=df.groupby(['DATEIN','PAYERACCOUNTNUMBER','PAYEEACCOUNTNUMBER'])['AMOUNT'].sum().reset_index() # Сначала группируем по датам, после начинаем группировать по плательщикам и получателям. Если поставить дату в конец, то будут проблемы

print (res)
set_with_dataframe(worksheet7, res) # записываем результат в отдельный лист

                DATEIN    PAYERACCOUNTNUMBER    PAYEEACCOUNTNUMBER     AMOUNT
0     01.01.2015 05:59  40817810300000000003  40817810300000000005   999000.0
1     01.01.2015 05:59  40817810300000000003     40917810351888632   999000.0
2     02.09.2019 10:08     20220810451876632     40817810451876632        0.0
3     03.08.2019 00:00  20202010400003155991  20202810105002300604  3957000.0
4     03.08.2019 00:00  20202010400003155991  20202811105002300604   600000.0
...                ...                   ...                   ...        ...
1245  31.07.2017 16:51  47402810940000003229  30220810940000003229  1600000.0
1246  31.07.2017 16:51  49901810099000418179  30317810142254000188   600000.0
1247  31.08.2017 14:55  40817810142254000188  42301810500230060440    15000.0
1248  31.08.2017 15:48  40817810142254000188  42306810516270008028    20868.0
1249  31.08.2017 16:46  40817810400003155991  40703810762420000003     3500.0

[1250 rows x 4 columns]


In [55]:
#Общая количество операций от плательщика x получателю y за дату z (для каждой пары плательщик-получатель и каждой даты)
clearing(worksheet8) # Чистим наш лист

res=df.groupby(['DATEIN','PAYERACCOUNTNUMBER','PAYEEACCOUNTNUMBER']).size().reset_index() # Получаем тут размер датафрейма

res.columns = ['DATEIN', 'PAYERACCOUNTNUMBER','PAYEEACCOUNTNUMBER','How many times'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet8, res) # записываем результат в отдельный лист

                DATEIN  ... How many times
0     01.01.2015 05:59  ...              1
1     01.01.2015 05:59  ...              1
2     02.09.2019 10:08  ...              1
3     03.08.2019 00:00  ...             13
4     03.08.2019 00:00  ...              1
...                ...  ...            ...
1245  31.07.2017 16:51  ...              2
1246  31.07.2017 16:51  ...              1
1247  31.08.2017 14:55  ...              2
1248  31.08.2017 15:48  ...              2
1249  31.08.2017 16:46  ...              2

[1250 rows x 4 columns]


In [57]:
#Общее сумма операций за дату z  (для каждой даты)
"""
Группируем по датам и суммируем данные
"""
clearing(worksheet9) # Чистим наш лист
res=df.groupby(['DATEIN'])['AMOUNT'].sum().apply(lambda x: '%.3f' % x).reset_index() # Использовал лямбду, чтобы числа показывались в читаемом формате (а не е+60)

res.columns = ['DATEIN','Total Sum'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet9, res) # записываем результат в отдельный лист

              DATEIN       Total Sum
0   01.01.2015 05:59     1998000.000
1   02.09.2019 10:08           0.000
2   03.08.2019 00:00  1226597351.190
3   05.03.2020 10:26     1998000.000
4   05.05.2018 20:59     1053965.990
..               ...             ...
61  31.07.2017 12:04    24723000.000
62  31.07.2017 16:51    27502000.000
63  31.08.2017 14:55       15000.000
64  31.08.2017 15:48       20868.000
65  31.08.2017 16:46        3500.000

[66 rows x 2 columns]


In [59]:
#Общее количество операций за дату z  (для каждой даты)
clearing(worksheet10) # Чистим наш лист

res=df.groupby(['DATEIN'])['AMOUNT'].size().reset_index() # Узнаем после группировки размер датафрейма

res.columns = ['DATEIN','How many times'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet10, res) # записываем результат в отдельный лист

              DATEIN  How many times
0   01.01.2015 05:59               2
1   02.09.2019 10:08               1
2   03.08.2019 00:00            2623
3   05.03.2020 10:26               2
4   05.05.2018 20:59               2
..               ...             ...
61  31.07.2017 12:04              39
62  31.07.2017 16:51              39
63  31.08.2017 14:55               2
64  31.08.2017 15:48               2
65  31.08.2017 16:46               2

[66 rows x 2 columns]


In [60]:
#Максимальная сумма отдельной операции каждого плательщика
clearing(worksheet11) # Чистим наш лист

res=df.groupby(['PAYERACCOUNTNUMBER'])['AMOUNT'].max().reset_index() # Путем вызова max мы узнаем максимальное значение для каждого поля 

res.columns = ['PAYERACCOUNTNUMBER','MAX sum'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet11, res) # записываем результат в отдельный лист

        PAYERACCOUNTNUMBER   MAX sum
0     20201810038320100992  999000.0
1     20202010400003155991  600000.0
2     20202121010001100111  610000.0
3     20202121010001100800  640100.0
4     20202810038320100992  999000.0
..                     ...       ...
366   91211810300000000003  999000.0
367   91217810400003155991  600000.0
368  912221210100010005756  500000.0
369   91312810000760001770  999000.0
370   99901810500230060440  600000.0

[371 rows x 2 columns]


In [62]:
#Максимальная сумма отдельно операции каждого получателя
clearing(worksheet12) # Чистим наш лист

res=df.groupby(['PAYEEACCOUNTNUMBER'])['AMOUNT'].max().reset_index() # сделал тоже самое, что и в предыдущей ячейке, только изменил плательщика на получателя

res.columns = ['PAYEEACCOUNTNUMBER','MAX sum'] # Изменил названия столбцов
print (res)
set_with_dataframe(worksheet12, res) # записываем результат в отдельный лист

       PAYEEACCOUNTNUMBER    MAX sum
0    20202010500230060440   18294.96
1    20202018105002300604  800000.00
2    20202181050023006044  600000.00
3    20202710038320100992  999000.00
4    20202809051627000802  600000.00
..                    ...        ...
440  98053810142254000188  100000.00
441  98055810940000003229  100000.00
442  99901810500230060440  600000.00
443  99902810300000000005  999000.00
444  99919710038320100992  999000.00

[445 rows x 2 columns]
